<a href="https://colab.research.google.com/github/dylannguyen2001/COS70008/blob/week3%2Fsentiment_analysis/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ==========================
# Choose ONE:
MODE = "roberta"        # "roberta" = 3-class sentiment (pos, neu, neg)
# MODE = "emotion"      # 7-class emotions (anger, joy, sadness, fear, disgust, surprise, neutral)
# ==========================

!pip -q install pandas pyarrow tqdm transformers torch

import os, time
import pandas as pd
import pyarrow.parquet as pq
import torch
import torch.nn.functional as F
from google.colab import files
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

print("CUDA:", torch.cuda.is_available(), "|",
      torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

# Pick model and output name
if MODE == "roberta":
    MODEL_NAME = "cardiffnlp/twitter-roberta-base-sentiment-latest"
    OUT_PATH   = "SentimentScores_roberta.parquet"
elif MODE == "emotion":
    MODEL_NAME = "j-hartmann/emotion-english-distilroberta-base"
    OUT_PATH   = "SentimentScores_emotion.parquet"
else:
    raise ValueError("MODE must be 'roberta' or 'emotion'")

BATCH_SIZE = 2048   # try 256 if GPU RAM allows
MAX_LENGTH = 128
LIMIT = None       # set to 1000 for a quick smoke test

# Upload only if files not already in workspace
need_upload = not (os.path.exists("TextBase.parquet") and os.path.exists("ThreadText.parquet"))
if need_upload:
    print("Please upload TextBase.parquet and ThreadText.parquet …")
    files.upload()

# Build pipeline
device = "cuda" if torch.cuda.is_available() else "cpu"
tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
mdl = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME).to(device)
id2label = getattr(mdl.config, "id2label", {})

def to_label(lbl):
    s = str(lbl)
    if s.lower().startswith("label_"):
        try:
            return id2label.get(int(s.split("_")[-1]), s)
        except:
            return s
    return s

def _pick_text_cols(parquet_path):
    cols = set(pq.ParquetFile(parquet_path).schema.names)
    subj_cands = ["subject_norm","subject","thread_subject","title","topic"]
    body_cands = ["body_clean","body_norm","body","text","content","message","thread_text"]
    subj = next((c for c in subj_cands if c in cols), None)
    body = next((c for c in body_cands if c in cols), None)
    print(f"[{parquet_path}] using subject={subj or '<empty>'}, body={body or '<empty>'}")
    if subj is None and body is None:
        raise ValueError(f"No suitable text columns found in {parquet_path}. Columns present: {sorted(list(cols))[:25]} ...")
    return subj, body

@torch.inference_mode()
def _score_batch(texts, max_length=128):
    enc = tok(texts, truncation=True, padding="max_length", max_length=max_length, return_tensors="pt")
    dev = "cuda" if torch.cuda.is_available() else "cpu"
    enc = {k: v.to(dev, non_blocking=True) for k, v in enc.items()}
    logits = mdl(**enc).logits
    probs = F.softmax(logits.float(), dim=-1)
    conf, idx = probs.max(dim=-1)
    labels = [id2label[int(i)] for i in idx]
    return labels, conf.tolist()


def run_one(path, id_col):
    subj_col, body_col = _pick_text_cols(path)
    text_cols = [col for col in [subj_col, body_col] if col is not None]

    pf = pq.ParquetFile(path)
    total = pf.metadata.num_rows if LIMIT is None else min(pf.metadata.num_rows, LIMIT)
    rows, seen, t0 = [], 0, time.time()
    for batch in pf.iter_batches(batch_size=BATCH_SIZE, columns=[id_col] + text_cols):
        df = batch.to_pandas()
        df[text_cols] = df[text_cols].fillna("")
        texts = (df[text_cols[0]] + (". " + df[text_cols[1]] if len(text_cols) > 1 else "")).str.strip().tolist()
        ids = df[id_col].tolist()

        labels, scores = _score_batch(texts, max_length=MAX_LENGTH)
        toks = tok(texts, truncation=True, max_length=MAX_LENGTH, return_length=True, padding=False)
        lens = toks["length"]

        for pid, label, score, L in zip(ids, labels, scores, lens):
            rows.append({
                "email_id":  pid if id_col == "email_id"  else None,
                "thread_id": pid if id_col == "thread_id" else None,
                "sentiment_label": to_label(label),
                "sentiment_score": float(score),
                "text_len_tokens": int(L),
            })

        seen += len(ids)
        elapsed = max(time.time() - t0, 1e-6)
        rps = seen / elapsed
        eta_min = max((total - seen) / rps, 0) / 60
        print(f"[{seen}/{total}] {seen/total*100:.1f}% | {rps:.1f} rows/s | ETA {eta_min:.1f} min")
        if LIMIT is not None and seen >= LIMIT:
            break
    return pd.DataFrame(rows)

dfs = []
if os.path.exists("TextBase.parquet"):
    print("Scoring emails …")
    dfs.append(run_one("TextBase.parquet", "email_id"))
else:
    print("TextBase.parquet not found, skipping")

if os.path.exists("ThreadText.parquet"):
    print("Scoring threads …")
    dfs.append(run_one("ThreadText.parquet", "thread_id"))
else:
    print("ThreadText.parquet not found, skipping")


out = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame(
    columns=["email_id","thread_id","sentiment_label","sentiment_score","text_len_tokens"]
)
out.to_parquet(OUT_PATH, index=False)
print(f"Done. Wrote {len(out):,} rows to {OUT_PATH}")

CUDA: True | NVIDIA A100-SXM4-40GB


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Scoring emails …
[TextBase.parquet] using subject=subject_norm, body=body_clean
[2048/245121] 0.8% | 336.9 rows/s | ETA 12.0 min
[4096/245121] 1.7% | 301.7 rows/s | ETA 13.3 min
[6144/245121] 2.5% | 316.1 rows/s | ETA 12.6 min
[8192/245121] 3.3% | 261.4 rows/s | ETA 15.1 min
[10240/245121] 4.2% | 251.4 rows/s | ETA 15.6 min
[12288/245121] 5.0% | 261.3 rows/s | ETA 14.9 min
[14336/245121] 5.8% | 269.4 rows/s | ETA 14.3 min
[16384/245121] 6.7% | 271.7 rows/s | ETA 14.0 min
[18432/245121] 7.5% | 277.8 rows/s | ETA 13.6 min
[20480/245121] 8.4% | 280.8 rows/s | ETA 13.3 min
[22528/245121] 9.2% | 281.4 rows/s | ETA 13.2 min
[24576/245121] 10.0% | 276.1 rows/s | ETA 13.3 min
[26624/245121] 10.9% | 272.6 rows/s | ETA 13.4 min
[28672/245121] 11.7% | 262.6 rows/s | ETA 13.7 min
[30720/245121] 12.5% | 255.2 rows/s | ETA 14.0 min
[32768/245121] 13.4% | 240.9 rows/s | ETA 14.7 min
[34816/245121] 14.2% | 231.2 rows/s | ETA 15.2 min
[36864/245121] 15.0% | 229.3 rows/s | ETA 15.1 min
[38912/245121] 15

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
# ==========================
# Emotion run (DistilRoBERTa)
# ==========================

!pip -q install pandas pyarrow tqdm transformers torch

import os, time
import pandas as pd
import pyarrow.parquet as pq
import torch, torch.nn.functional as F
from google.colab import files
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_NAME = "j-hartmann/emotion-english-distilroberta-base"  # emotions
OUT_PATH   = "SentimentScores_emotion.parquet"
BATCH_SIZE = 2048      # safe start on A100. Try 1536 or 2048 if plenty of VRAM
MAX_LENGTH = 128
LIMIT      = None      # set to 1000 for a quick smoke test

print("CUDA:", torch.cuda.is_available(), "|",
      torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

# Use existing uploads in this session
if not (os.path.exists("TextBase.parquet") and os.path.exists("ThreadText.parquet")):
    print("Please upload TextBase.parquet and ThreadText.parquet …")
    files.upload()

tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
mdl = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, torch_dtype=torch.bfloat16 if torch.cuda.is_available() else None
).to("cuda" if torch.cuda.is_available() else "cpu").eval()
id2label = mdl.config.id2label

def _pick_text_cols(parquet_path):
    cols = set(pq.ParquetFile(parquet_path).schema.names)
    subj_cands = ["subject_norm","subject","thread_subject","title","topic"]
    body_cands = ["body_clean","body_norm","body","text","content","message","thread_text"]
    subj = next((c for c in subj_cands if c in cols), None)
    body = next((c for c in body_cands if c in cols), None)
    print(f"[{parquet_path}] using subject={subj or '<empty>'}, body={body or '<empty>'}")
    if subj is None and body is None:
        raise ValueError(f"No suitable text columns found in {parquet_path}.")
    return subj, body

@torch.inference_mode()
def _score_batch(texts, max_length=128):
    enc = tok(texts, truncation=True, padding="max_length", max_length=max_length, return_tensors="pt")
    dev = "cuda" if torch.cuda.is_available() else "cpu"
    enc = {k: v.to(dev, non_blocking=True) for k, v in enc.items()}
    logits = mdl(**enc).logits
    probs = F.softmax(logits.float(), dim=-1)
    conf, idx = probs.max(dim=-1)
    labels = [id2label[int(i)] for i in idx]
    return labels, conf.tolist()

def run_one(path, id_col):
    subj_col, body_col = _pick_text_cols(path)
    text_cols = [c for c in [subj_col, body_col] if c is not None]

    pf = pq.ParquetFile(path)
    total = pf.metadata.num_rows if LIMIT is None else min(pf.metadata.num_rows, LIMIT)
    rows, seen, t0 = [], 0, time.time()

    for batch in pf.iter_batches(batch_size=BATCH_SIZE, columns=[id_col] + text_cols):
        df = batch.to_pandas()
        df[text_cols] = df[text_cols].fillna("")
        texts = (df[text_cols[0]] + (". " + df[text_cols[1]] if len(text_cols) > 1 else "")).str.strip().tolist()
        ids = df[id_col].tolist()

        labels, scores = _score_batch(texts, max_length=MAX_LENGTH)
        toks = tok(texts, truncation=True, max_length=MAX_LENGTH, return_length=True, padding=False)
        lens = toks["length"]

        for pid, lab, scr, L in zip(ids, labels, scores, lens):
            rows.append({
                "email_id":  pid if id_col == "email_id"  else None,
                "thread_id": pid if id_col == "thread_id" else None,
                "sentiment_label": lab,          # emotion label here
                "sentiment_score": float(scr),   # confidence
                "text_len_tokens": int(L),
            })

        seen += len(ids)
        rps = seen / max(time.time() - t0, 1e-6)
        eta_min = max((total - seen) / rps, 0) / 60
        print(f"[{seen}/{total}] {seen/total*100:.1f}% | {rps:.1f} rows/s | ETA {eta_min:.1f} min")
        if LIMIT is not None and seen >= LIMIT: break

    return pd.DataFrame(rows)

dfs = []
if os.path.exists("TextBase.parquet"):
    print("Scoring emails …")
    dfs.append(run_one("TextBase.parquet", "email_id"))
if os.path.exists("ThreadText.parquet"):
    print("Scoring threads …")
    dfs.append(run_one("ThreadText.parquet", "thread_id"))

out = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame(
    columns=["email_id","thread_id","sentiment_label","sentiment_score","text_len_tokens"]
)
out.to_parquet(OUT_PATH, index=False)
print(f"Done. Wrote {len(out):,} rows to {OUT_PATH}")
files.download(OUT_PATH)


CUDA: True | NVIDIA A100-SXM4-40GB


tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

Scoring emails …
[TextBase.parquet] using subject=subject_norm, body=body_clean


model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

[2048/245121] 0.8% | 525.8 rows/s | ETA 7.7 min
[4096/245121] 1.7% | 448.4 rows/s | ETA 9.0 min
[6144/245121] 2.5% | 473.0 rows/s | ETA 8.4 min
[8192/245121] 3.3% | 368.7 rows/s | ETA 10.7 min
[10240/245121] 4.2% | 350.5 rows/s | ETA 11.2 min
[12288/245121] 5.0% | 369.6 rows/s | ETA 10.5 min
[14336/245121] 5.8% | 387.4 rows/s | ETA 9.9 min
[16384/245121] 6.7% | 393.1 rows/s | ETA 9.7 min
[18432/245121] 7.5% | 406.9 rows/s | ETA 9.3 min
[20480/245121] 8.4% | 413.8 rows/s | ETA 9.0 min
[22528/245121] 9.2% | 413.0 rows/s | ETA 9.0 min
[24576/245121] 10.0% | 402.0 rows/s | ETA 9.1 min
[26624/245121] 10.9% | 397.3 rows/s | ETA 9.2 min
[28672/245121] 11.7% | 377.0 rows/s | ETA 9.6 min
[30720/245121] 12.5% | 362.3 rows/s | ETA 9.9 min
[32768/245121] 13.4% | 333.1 rows/s | ETA 10.6 min
[34816/245121] 14.2% | 314.7 rows/s | ETA 11.1 min
[36864/245121] 15.0% | 311.8 rows/s | ETA 11.1 min
[38912/245121] 15.9% | 308.2 rows/s | ETA 11.2 min
[40960/245121] 16.7% | 304.4 rows/s | ETA 11.2 min
[43008/

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
import pandas as pd
import pyarrow.parquet as pq
from collections import Counter

def info_parquet(path):
    pf = pq.ParquetFile(path)
    print(f"\n=== {path} ===")
    print("rows:", pf.metadata.num_rows)
    print("columns:", pf.schema.names)
    df = pd.read_parquet(path)
    # Basic schema check
    needed = {"sentiment_label","sentiment_score","text_len_tokens"}
    assert needed.issubset(df.columns), f"{path} missing {needed - set(df.columns)}"
    assert "email_id" in df.columns or "thread_id" in df.columns, f"{path} missing id column"
    # Dtypes and nulls
    print(df.dtypes)
    print("nulls:\n", df.isna().sum())
    # Label distribution
    print("label_counts:\n", df["sentiment_label"].value_counts().head(10))
    # Score sanity
    print("score range:", float(df["sentiment_score"].min()), "→", float(df["sentiment_score"].max()))
    # Token length sanity
    print("token_len stats:\n", df["text_len_tokens"].describe())
    # Show a few rows
    display(df.head(5))
    return df

# Load what you produced
paths = []
try:
    open("SentimentScores_roberta.parquet"); paths.append("SentimentScores_roberta.parquet")
except: pass
try:
    open("SentimentScores_emotion.parquet"); paths.append("SentimentScores_emotion.parquet")
except: pass

assert paths, "No sentiment parquet found in the workspace."

dfs = {p: info_parquet(p) for p in paths}

# Optional: cross-check overlap between roberta and emotion on IDs
if len(dfs) == 2:
    r_df = dfs["SentimentScores_roberta.parquet"].copy()
    e_df = dfs["SentimentScores_emotion.parquet"].copy()
    # Build a common key preferring email_id else thread_id
    r_df["key"] = r_df["email_id"].fillna(r_df["thread_id"])
    e_df["key"] = e_df["email_id"].fillna(e_df["thread_id"])
    # Drop duplicates if any
    r_df = r_df.drop_duplicates(["key"])
    e_df = e_df.drop_duplicates(["key"])
    joined = r_df.merge(e_df[["key","sentiment_label","sentiment_score"]], on="key", how="inner", suffixes=("_polarity","_emotion"))
    print("\n=== Join check (polarity vs emotion) ===")
    print("overlap rows:", len(joined), "of", len(r_df), "(roberta unique)")
    display(joined.head(10))
    # Quick contingency of polarity vs emotion
    c = Counter(zip(joined["sentiment_label_polarity"], joined["sentiment_label_emotion"]))
    top = pd.DataFrame([{"polarity":k[0],"emotion":k[1],"count":v} for k,v in c.items()]).sort_values("count", ascending=False).head(12)
    print("\nTop co-occurrences:")
    display(top)

# Save a tiny sample for manual review
for p, df in dfs.items():
    sample_path = p.replace(".parquet","_sample.csv")
    df.head(50).to_csv(sample_path, index=False)
    print("wrote sample:", sample_path)



=== SentimentScores_roberta.parquet ===
rows: 436755
columns: ['email_id', 'thread_id', 'sentiment_label', 'sentiment_score', 'text_len_tokens']
email_id            object
thread_id           object
sentiment_label     object
sentiment_score    float64
text_len_tokens      int64
dtype: object
nulls:
 email_id           191634
thread_id          245121
sentiment_label         0
sentiment_score         0
text_len_tokens         0
dtype: int64
label_counts:
 sentiment_label
neutral     374920
positive     46657
negative     15178
Name: count, dtype: int64
score range: 0.3394984304904938 → 0.9907373785972595
token_len stats:
 count    436755.000000
mean         62.236556
std          54.841762
min           2.000000
25%           9.000000
50%          38.000000
75%         128.000000
max         128.000000
Name: text_len_tokens, dtype: float64


,email_id,thread_id,sentiment_label,sentiment_score,text_len_tokens
0,84ff6ceda84132773b8e1f3c87a5cfc2,None,neutral,0.892314,128
1,f8eb6168a76187937c662c46431f635e,None,neutral,0.890933,128
2,804fc5132f645eec7a2fc67fd8ed973f,None,neutral,0.893232,128
3,01cfb1b50f164d8e427e54459f236dd7,None,neutral,0.784084,34
4,d6dc5fd8e7846e5f820410c3201d0161,None,neutral,0.637044,17



=== SentimentScores_emotion.parquet ===
rows: 436755
columns: ['email_id', 'thread_id', 'sentiment_label', 'sentiment_score', 'text_len_tokens']
email_id            object
thread_id           object
sentiment_label     object
sentiment_score    float64
text_len_tokens      int64
dtype: object
nulls:
 email_id           191634
thread_id          245121
sentiment_label         0
sentiment_score         0
text_len_tokens         0
dtype: int64
label_counts:
 sentiment_label
neutral     362452
joy          22118
sadness      18404
surprise     16162
fear         11483
anger         4983
disgust       1153
Name: count, dtype: int64
score range: 0.19321288168430328 → 0.9945107698440552
token_len stats:
 count    436755.000000
mean         62.236556
std          54.841762
min           2.000000
25%           9.000000
50%          38.000000
75%         128.000000
max         128.000000
Name: text_len_tokens, dtype: float64


,email_id,thread_id,sentiment_label,sentiment_score,text_len_tokens
0,84ff6ceda84132773b8e1f3c87a5cfc2,None,neutral,0.739193,128
1,f8eb6168a76187937c662c46431f635e,None,neutral,0.718593,128
2,804fc5132f645eec7a2fc67fd8ed973f,None,neutral,0.718075,128
3,01cfb1b50f164d8e427e54459f236dd7,None,neutral,0.735880,34
4,d6dc5fd8e7846e5f820410c3201d0161,None,neutral,0.500213,17



=== Join check (polarity vs emotion) ===
overlap rows: 436755 of 436755 (roberta unique)


,email_id,thread_id,sentiment_label_polarity,sentiment_score_polarity,text_len_tokens,key,sentiment_label_emotion,sentiment_score_emotion
0,84ff6ceda84132773b8e1f3c87a5cfc2,None,neutral,0.892314,128,84ff6ceda84132773b8e1f3c87a5cfc2,neutral,0.739193
1,f8eb6168a76187937c662c46431f635e,None,neutral,0.890933,128,f8eb6168a76187937c662c46431f635e,neutral,0.718593
2,804fc5132f645eec7a2fc67fd8ed973f,None,neutral,0.893232,128,804fc5132f645eec7a2fc67fd8ed973f,neutral,0.718075
3,01cfb1b50f164d8e427e54459f236dd7,None,neutral,0.784084,34,01cfb1b50f164d8e427e54459f236dd7,neutral,0.735880
4,d6dc5fd8e7846e5f820410c3201d0161,None,neutral,0.637044,17,d6dc5fd8e7846e5f820410c3201d0161,neutral,0.500213
5,56664254dd20bcf4d4b611a8617e0bcb,None,neutral,0.924952,128,56664254dd20bcf4d4b611a8617e0bcb,neutral,0.553381
6,c13d1d7b5916af6c178bb6f28e43b5b3,None,neutral,0.935403,128,c13d1d7b5916af6c178bb6f28e43b5b3,neutral,0.845079
7,2bbd3b5872f999d8bb368957130de081,None,neutral,0.938351,128,2bbd3b5872f999d8bb368957130de081,neutral,0.647372
8,fa87ffb72df40d3199a6570953375d69,None,neutral,0.889591,42,fa87ffb72df40d3199a6570953375d69,neutral,0.865198
9,2a2c1b445994fdff9d8c5521580abd2b,None,positive,0.652788,24,2a2c1b445994fdff9d8c5521580abd2b,neutral,0.922732



Top co-occurrences:


,polarity,emotion,count
0,neutral,neutral,329671
1,positive,neutral,26934
4,positive,joy,14127
5,neutral,sadness,12248
3,neutral,surprise,11651
2,neutral,fear,9540
9,neutral,joy,7825
8,negative,neutral,5847
7,negative,sadness,4868
12,neutral,anger,3319


wrote sample: SentimentScores_roberta_sample.csv
wrote sample: SentimentScores_emotion_sample.csv


In [16]:
import pandas as pd

files_to_check = [
    "SentimentScores_roberta.parquet",
    "SentimentScores_emotion.parquet"
]

expected_cols = ["email_id", "thread_id", "sentiment_label", "sentiment_score", "text_len_tokens"]

for f in files_to_check:
    try:
        df = pd.read_parquet(f)
        print(f"\n=== {f} ===")
        print("Shape:", df.shape)
        print("Columns:", list(df.columns))

        # Check required schema
        if all(c in df.columns for c in expected_cols):
            print("✅ Schema OK")
        else:
            missing = [c for c in expected_cols if c not in df.columns]
            print("❌ Missing columns:", missing)

        # Quick stats
        print("Null counts:\n", df.isna().sum())
        print("Label sample:", df["sentiment_label"].unique()[:10])
        print("Score range:", df["sentiment_score"].min(), "→", df["sentiment_score"].max())
        print("Token stats:\n", df["text_len_tokens"].describe())
        print(df.head(3))
    except FileNotFoundError:
        print(f"{f} not found in workspace")



=== SentimentScores_roberta.parquet ===
Shape: (131072, 5)
Columns: ['email_id', 'thread_id', 'sentiment_label', 'sentiment_score', 'text_len_tokens']
✅ Schema OK
Null counts:
 email_id           65536
thread_id          65536
sentiment_label        0
sentiment_score        0
text_len_tokens        0
dtype: int64
Label sample: ['neutral' 'positive' 'negative']
Score range: 0.34698185324668884 → 0.9903135299682617
Token stats:
 count    131072.000000
mean        274.505905
std        1072.142773
min           0.000000
25%          45.000000
50%         111.000000
75%         252.000000
max      123680.000000
Name: text_len_tokens, dtype: float64
                           email_id thread_id sentiment_label  \
0  84ff6ceda84132773b8e1f3c87a5cfc2      None         neutral   
1  f8eb6168a76187937c662c46431f635e      None         neutral   
2  804fc5132f645eec7a2fc67fd8ed973f      None         neutral   

   sentiment_score  text_len_tokens  
0         0.892422             1073  
1        

In [19]:
import pandas as pd

# load whichever sentiment file you want to check
df = pd.read_parquet("SentimentScores_emotion.parquet")

# filter rows where thread_id is not None/NaN
threads = df[df["thread_id"].notna()]

print("Total rows with thread_id:", len(threads))
print("Example rows with thread_id:\n")
print(threads.head(10))


Total rows with thread_id: 191634
Example rows with thread_id:

       email_id                         thread_id sentiment_label  \
245121     None  000015a939797cfcd09fae96e6f52b38        surprise   
245122     None  00004c897aba6dbc6f5113b4070bd471         sadness   
245123     None  00006aade04fa7906c13155acea00187        surprise   
245124     None  0000a5fdc9080c6101513e5153588b88            fear   
245125     None  000199ee291eeaf87abb9f44415ad2a6         neutral   
245126     None  000247de21255452ab7f1dca9cfbc4f9         neutral   
245127     None  00026d6455dc2782c59df1b88e2ad949         neutral   
245128     None  0003f095450a4a3e5eccaca7685f773b        surprise   
245129     None  00043dcfe6232f238a2a66d399822425         neutral   
245130     None  0004eceab02bd98ff90b7599add9ef1b         neutral   

        sentiment_score  text_len_tokens  
245121         0.407832                8  
245122         0.915437               12  
245123         0.712193               10  
2451

In [22]:
import pandas as pd

df = pd.read_parquet("SentimentScores_combined.parquet")

# rows where BOTH ids are missing
bad_rows = df[df["email_id"].isna() & df["thread_id"].isna()]

print("Rows with BOTH email_id and thread_id missing:", len(bad_rows))
if len(bad_rows) > 0:
    print("Example rows:\n", bad_rows.head(10))


Rows with BOTH email_id and thread_id missing: 0


In [6]:
import pandas as pd

POL_PATH = "SentimentScores_roberta.parquet"    # pos/neu/neg
EMO_PATH = "SentimentScores_emotion.parquet"    # anger/joy/...
OUT_PATH = "SentimentScores_combined.parquet"

# ---------- Load ----------
pol = pd.read_parquet(POL_PATH)
emo = pd.read_parquet(EMO_PATH)

# Build a common key: prefer email_id, else thread_id
pol = pol.copy()
emo = emo.copy()
pol["key"] = pol["email_id"].fillna(pol["thread_id"])
emo["key"] = emo["email_id"].fillna(emo["thread_id"])

# ---------- Deduplicate by best confidence ----------
# If multiple rows share the same key (e.g., aggregated differently),
# keep the one with the highest sentiment_score within each file.
pol = pol.sort_values("sentiment_score", ascending=False).drop_duplicates("key", keep="first")
emo = emo.sort_values("sentiment_score", ascending=False).drop_duplicates("key", keep="first")

# ---------- Merge ----------
# Use outer join so keys present in only one file are still included.
combined = pol.merge(
    emo[["key", "sentiment_label", "sentiment_score"]],
    on="key", how="outer",
    suffixes=("_polarity", "_emotion")
)

# Bring email_id/thread_id back (prefer those from polarity side if present)
combined["email_id"] = combined["email_id"].fillna(combined.pop("thread_id"))  # temporary trick to reuse the column
# The line above moved thread_id into email_id for rows missing email_id.
# Let's reconstruct cleanly:
combined["thread_id"] = None
# If your data truly has either email_id or thread_id exclusively, you can leave it like this.
# Otherwise, if you want to strictly keep both original id columns:
# - Read them again from source and merge; or keep separate before the above trick.

# For clarity, rebuild ids more explicitly:
def split_ids(row):
    eid = row.get("email_id", None)
    # If what we filled came originally from a thread_id, it won't be an email id.
    # If your inputs always had exactly one id per row (email OR thread) this is fine.
    return pd.Series({"email_id": eid if pd.notna(eid) else None,
                      "thread_id": None})

ids = combined.apply(split_ids, axis=1)
combined["email_id"] = ids["email_id"]
combined["thread_id"] = ids["thread_id"]

# Keep a token length column (prefer polarity’s if available, else emotion’s)
if "text_len_tokens_polarity" in combined.columns:
    pass
else:
    # rename if needed because we did not suffix this column earlier
    combined.rename(columns={"text_len_tokens":"text_len_tokens_polarity"}, inplace=True)

# If emotion file had token lengths and polarity didn’t, you can adapt similarly.
if "text_len_tokens_polarity" in combined.columns:
    combined["text_len_tokens"] = combined["text_len_tokens_polarity"]
elif "text_len_tokens_emotion" in combined.columns:
    combined["text_len_tokens"] = combined["text_len_tokens_emotion"]
else:
    combined["text_len_tokens"] = None

# Final tidy columns
final_cols = [
    "email_id",
    "thread_id",
    "sentiment_label_polarity",
    "sentiment_score_polarity",
    "sentiment_label_emotion",
    "sentiment_score_emotion",
    "text_len_tokens",
]
# Ensure these exist (create if missing)
for c in final_cols:
    if c not in combined.columns:
        combined[c] = None

combined = combined[final_cols]

# ---------- Save ----------
combined.to_parquet(OUT_PATH, index=False)
combined.head(50).to_csv("SentimentScores_combined_sample.csv", index=False)
print(f"✅ Combined rows: {len(combined):,}")
print("Wrote:", OUT_PATH, "and SentimentScores_combined_sample.csv")


✅ Combined rows: 436,755
Wrote: SentimentScores_combined.parquet and SentimentScores_combined_sample.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>